In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import AutoTokenizer, RobertaTokenizer, AdamWeightDecay, TFAutoModel
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordTokenizer
from sklearn.model_selection import KFold
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [4]:
# Download NLTK resources
nltk.download('punkt')  # Download the NLTK resource for tokenization

# Unzip WordNet data file to the NLTK data directory
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
# WordNet is a lexical database for the English language. This command unzips its data to the appropriate NLTK data directory.

nltk.download('wordnet')  # Download the NLTK resource for WordNet
# WordNet is a large lexical database of English. It's used for various NLP tasks, including lemmatization and synonym/antonym retrieval.

nltk.download('stopwords')  # Download the NLTK resource for stopwords
# Stopwords are common words (e.g., "the," "and," "is") that are often removed from text data when performing natural language processing tasks.

stop_words = set(stopwords.words('english'))  # Create a set of English stopwords
# This line initializes a set containing common English stopwords, which can be used for text preprocessing.

lemmatizer = WordNetLemmatizer()  # Initialize a WordNet lemmatizer
# The WordNet lemmatizer is used to reduce words to their base or dictionary form. For example, "running" would be lemmatized to "run."


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpor

In [5]:
dataset=pd.read_csv("/kaggle/input/cumulative-reviews-and-ratings/cumulative.csv")
dataset = dataset[dataset['rating'] != '|']
#make 100000 instead if 10000
dataset = dataset.sample(n=10000, random_state=42) #Taking a sample for training, otherwise kernel crashes, ig for offline implementation, this shouldn't be an issue

/tmp/ipykernel_32/1344748012.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv("/kaggle/input/cumulative-reviews-and-ratings/cumulative.csv")


In [6]:
print(type(dataset['review'].iloc[0]))

<class 'str'>


In [7]:
dataset['review'] = dataset['review'].str.replace('&', 'and')

def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    return re.sub(url_pattern, '', text)

# Apply the remove_urls function to the 'review' column
dataset['review'] = dataset['review'].apply(lambda x: remove_urls(x))

def remove_non_alphanumeric(text):
    return re.sub(r'[^a-zA-Z0-9.]', ' ', text)

# Apply the remove_non_alphanumeric function to the "reviews" column
dataset['review'] = dataset['review'].apply(remove_non_alphanumeric)

print(list(dataset.columns))
print(dataset.head())
print(len(dataset))

['rating', 'review']
       rating                                             review
303990      5  My son and I love these beef sticks  I purchas...
269665      5  Since the stores do not carry this product  I ...
380357      5  My cat loves all the Weruva varieties. I rotat...
37894       5  I can have about 3 of these a week as a protei...
444547      5  HAVE BEEN USING THE CROWN PRICE LUMP WHITE CRA...
10000


In [8]:
dataset['rating'] = dataset['rating'].astype(float).round() #converting ratings to numerical values
dataset['review'] = dataset['review'].str.lower() #uniformly making all letters lower case in the reviews

print(list(dataset.columns))
print(dataset.head())
print(len(dataset))

['rating', 'review']
        rating                                             review
303990     5.0  my son and i love these beef sticks  i purchas...
269665     5.0  since the stores do not carry this product  i ...
380357     5.0  my cat loves all the weruva varieties. i rotat...
37894      5.0  i can have about 3 of these a week as a protei...
444547     5.0  have been using the crown price lump white cra...
10000


In [9]:

def lemmatize_and_remove_stopwords(text):
    words = nltk.word_tokenize(text) 
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the lemmatize_and_remove_stopwords function to the "reviews" column
dataset['review'] = dataset['review'].apply(lemmatize_and_remove_stopwords)

print(list(dataset.columns))
print(dataset.head())
print(len(dataset))


['rating', 'review']
        rating                                             review
303990     5.0  son love beef stick purchased slim jims time a...
269665     5.0  since store carry product lucky come across li...
380357     5.0  cat love weruva variety . rotate wellness wet ...
37894      5.0  3 week protein meal snack anytime feeling hung...
444547     5.0  using crown price lump white crab meat years.i...
10000


In [10]:
q=list(dataset['review'].isna())
print(sum(q))

0


In [11]:
# #LOOK INTO THIS MAX_LENGTH AND WHAT ARE THE PARAMETERS I AM USING
# def to_tokens(input_text, tokenizer):
#     output = tokenizer.encode_plus(text=input_text, max_length=90, pad_to_max_length=True, truncation=True)
# #     print("to_tokens:", type(output))
# #     print(output.keys())
#     return output

# def select_field(features, field):
#     return [feature[field] for feature in features]

In [12]:
# string="I love NLP, I need to know more about it. He is happy where he is."
# print(string)
# string = remove_non_alphanumeric(string)
# print(string)
# string = lemmatize_and_remove_stopwords(string)
# string=string.lower()
# print(string)

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# out = to_tokens(string,tokenizer)

# print(len(out['input_ids']),len(out['attention_mask']))
# print(out['input_ids'])
# print(out['attention_mask'])

# print(tokenizer.convert_ids_to_tokens(out['input_ids']))

input_text = "I love natural language processing"

simplified output =
{
    'input_ids': [0, 101, 1045, 2293, 3015, 2652, 3836, 102, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
}

In [13]:
# def preprocess_data(tokenizer):
#     data_encoded = dataset['review'].apply(lambda x: to_tokens(x, tokenizer))

#     # Create attention masks
#     input_ids = np.array(select_field(data_encoded, 'input_ids'))
#     attention_masks = np.array(select_field(data_encoded, 'attention_mask'))
    
#     # Manually split data into train and test sets
#     split_index = int(len(data_encoded) * 0.7)
#     X_train_input_ids, X_test_input_ids = input_ids[:split_index], input_ids[split_index:]
#     X_train_attention_masks, X_test_attention_masks = attention_masks[:split_index], attention_masks[split_index:]
    
#     X_train = np.array([X_train_input_ids, X_train_attention_masks])
#     X_test = np.array([X_test_input_ids, X_test_attention_masks])
    
#     y_train = np.array(dataset['rating'].iloc[:split_index])
#     y_test = np.array(dataset['rating'].iloc[split_index:])
    
#     return X_train, X_test, y_train, y_test


In [14]:
def load(learning_rate,epsilon=1e-8,num_labels=5,task="sentiment_analysis",model_name='roberta-large'):
    config_class, model_class, tokenizer_class= RobertaConfig, TFRobertaForSequenceClassification, RobertaTokenizer

    config = config_class.from_pretrained(model_name, num_labels=num_labels, finetuning_task="multi_class")


    model_metrics = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'), 
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
    ]

    model = model_class.from_pretrained(model_name)
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon, clipnorm=1.0)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    metric = keras.metrics.CategoricalAccuracy('accuracy')

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    tokenizer = tokenizer_class.from_pretrained(model_name, lower_case = False)
    return config, model, tokenizer

In [15]:
# config, model, tokenizer = load(learning_rate=2e-5)
# X_train,X_test,y_train,y_test=preprocess_data(tokenizer=tokenizer)

In [16]:
train = dataset.iloc[:int(dataset.shape[0] * 0.7)]
test = dataset.iloc[int(dataset.shape[0] * 0.7):]
print(train.shape,test.shape)
x_train,y_train = train['review'],train['rating']
x_test, y_test = test['review'],test['rating']
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(7000, 2) (3000, 2)
(7000,) (7000,)
(3000,) (3000,)


In [17]:
tokenizer =  RobertaTokenizer.from_pretrained("roberta-base")

def tokenize(df):
    input_ids =  []
    attention_masks =  []
    
    for i, text in enumerate(df):
        tokens = tokenizer.encode_plus(text, max_length=70,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')
         
        input_ids.append(np.asarray(tokens["input_ids"]).reshape(70,))
        attention_masks.append(np.asarray(tokens["attention_mask"]).reshape(70,))

    return (np.asarray(input_ids), np.asarray(attention_masks))

x_train_input_ids, x_train_attention_masks = tokenize(x_train)
x_test_input_ids, x_test_attention_masks = tokenize(x_test)


In [18]:
print(x_train_input_ids.shape, x_train_attention_masks.shape)
print(x_test_input_ids.shape, x_test_attention_masks.shape)

(7000, 70) (7000, 70)
(3000, 70) (3000, 70)


In [19]:
def one_encode_labels(df):
    sentiment_values = set(dataset["rating"].values)
    labels = []
    for index, row in df.iterrows():
        label = np.zeros((len(sentiment_values)))
        label[row["sentiment"]] = 1 
        labels.append(label)
    labels = np.asarray(labels)
    return labels

In [20]:
print(y_train,y_test)

303990    5.0
269665    5.0
380357    5.0
37894     5.0
444547    5.0
         ... 
64465     1.0
61725     4.0
160540    5.0
345485    2.0
8700      5.0
Name: rating, Length: 7000, dtype: float64 88638     1.0
242005    5.0
12029     5.0
230383    1.0
230623    4.0
         ... 
419306    5.0
121499    5.0
259270    4.0
294861    1.0
251404    4.0
Name: rating, Length: 3000, dtype: float64


In [21]:
print(y_train)

303990    5.0
269665    5.0
380357    5.0
37894     5.0
444547    5.0
         ... 
64465     1.0
61725     4.0
160540    5.0
345485    2.0
8700      5.0
Name: rating, Length: 7000, dtype: float64


In [22]:
def one_encode_labels(df):
    sentiment_values = set(df)
    labels = []
    for rating in df:
        label = np.zeros((len(sentiment_values)))
        if rating == 1.0:
            label[0]=1
        elif rating == 2.0:
            label[1] =1
        elif rating ==3.0:
            label[2]=1
        elif rating == 4.0:
            label[3]=1
        elif rating == 5.0:
            label[4] =1
        labels.append(label)
    labels = np.asarray(labels)
    return labels

y_train = one_encode_labels(y_train)
y_test = one_encode_labels(y_test)


In [23]:
roberta = TFAutoModel.from_pretrained("roberta-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [24]:
input_ids = tf.keras.layers.Input(shape=(70,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(70,), name='attention_mask', dtype='int32')

embeddings = roberta(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.LSTM(128)(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(768)(X)
X = tf.keras.layers.Activation("relu")(X)
X = tf.keras.layers.Dense(768)(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 70)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 70)]         0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 70,                                            

In [25]:
optimizer = AdamWeightDecay(2e-03, beta_1=0.8, beta_2=0.9, weight_decay_rate=0.0001)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [26]:
print(x_train_input_ids.shape, x_train_attention_masks.shape)
print(x_test_input_ids.shape,x_test_attention_masks.shape)
print(y_train.shape,y_test.shape)

(7000, 70) (7000, 70)
(3000, 70) (3000, 70)
(7000, 5) (3000, 5)


In [27]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


In [30]:
history = model.fit((x_train_input_ids, x_train_attention_masks), y_train, validation_data=((x_test_input_ids, x_test_attention_masks), y_test), epochs=50,callbacks=[early_stopping])


Epoch 1/50
219/219 [==============================] - 53s 171ms/step - loss: 1.1381 - accuracy: 0.6327 - val_loss: 1.1249 - val_accuracy: 0.6513
Epoch 2/50
219/219 [==============================] - 33s 151ms/step - loss: 0.9494 - accuracy: 0.6646 - val_loss: 1.0021 - val_accuracy: 0.6713
Epoch 3/50
219/219 [==============================] - 33s 150ms/step - loss: 0.9098 - accuracy: 0.6731 - val_loss: 1.4311 - val_accuracy: 0.6620
Epoch 4/50
219/219 [==============================] - 33s 152ms/step - loss: 0.8752 - accuracy: 0.6817 - val_loss: 0.9141 - val_accuracy: 0.6723
Epoch 5/50
219/219 [==============================] - 33s 152ms/step - loss: 0.8520 - accuracy: 0.6867 - val_loss: 1.1495 - val_accuracy: 0.6760
Epoch 6/50
219/219 [==============================] - 33s 151ms/step - loss: 0.8422 - accuracy: 0.6907 - val_loss: 1.3350 - val_accuracy: 0.6190
Epoch 7/50
219/219 [==============================] - 33s 149ms/step - loss: 0.8325 - accuracy: 0.6967 - val_loss: 1.1598 - val_ac

In [ ]:
tokenizer =  RobertaTokenizer.from_pretrained("roberta-base")

# predictions need to go through same preprocessings
def prep_data(text):
    text = remove_urls(text)
    
    text = remove_non_alphanumeric(text)
    
    text = lemmatize_and_remove_stopwords(text)

    tokens = tokenizer.encode_plus(text, max_length=70,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')

    return {'input_ids': tf.cast(tokens['input_ids'], tf.float64),
            'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)}

def predict(text):
    in_tensor = prep_data(text)
    probs = model.predict(in_tensor)[0]
    return (np.argmax(probs))

In [ ]:
predictions = np.asarray(x_test.apply(lambda x: predict(x)))
test_vals = np.asarray(y_test)

In [ ]:
print(sum)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(test_vals, predictions)

# plot confusion matrix
sns.set(font_scale=1.0)
labels = ["1","2","3","4","5"]
ax = sns.heatmap(confusion, annot=True, annot_kws={"size": 11}, fmt='d', vmin = 0, cmap='Blues', yticklabels=labels, xticklabels=labels)
ax.set_xlabel('Predicted Class')   
ax.set_ylabel('True Class')   
ax.xaxis.set_label_position('top')
ax.xaxis.tick_top()
plt.show()

# print classification report
report = classification_report(test_vals, predictions, digits = 4, output_dict=False, target_names=["bad", "neutral", "good"],)
print(report)

In [ ]:
# kf = KFold(n_splits=6)
# test_preds = []
# i = 0
# for train_idx, test_idx in kf.split(X_train[0]):
#     i += 1
#     if i not in [1, 5]:  # only do 2 folds to save time
#         continue
#     train_split_X = [X_train[i][train_idx] for i in range(len(X_train))]
#     test_split_X = [X_train[i][test_idx] for i in range(len(X_train))]

#     train_split_y = y_train[train_idx]
#     test_split_y = y_train[test_idx]
#     # create class weights to account for imbalance
#     sentiment_counts = dataset.iloc[train_idx, :].target.value_counts()
#     total_samples = sentiment_counts.sum()

#     class_weights = {i: total_samples / sentiment_counts[i] for i in range(5)}
#     class_weights = {key: val / sum(class_weights.values()) for key, val in class_weights.items()}

#     K.clear_session()
#     config, model, tokenizer = load(learning_rate=2e-5)

#     # fit, test model
#     model.fit(train_split_X, train_split_y, batch_size=64, epochs=3, class_weight=class_weights, validation_data=(test_split_X, test_split_y))

#     val_preds = model.predict(test_split_X, batch_size=32, verbose=1)
#     val_preds = np.argmax(val_preds, axis=1).flatten()
#     print(metrics.accuracy_score(y_test[test_idx], val_preds))

#     preds1 = model.predict(X_test, batch_size=32, verbose=1)
#     test_preds.append(preds1)
